In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext wurlitzer

In [3]:
import pandas as pd
import numpy as np
import logging
logging.basicConfig(level = logging.DEBUG)

In [4]:
import pyfdb
import findlibs
import yaml
from pathlib import Path
import os
import pandas as pd
import pyodc
import shutil

In [5]:
from pathlib import Path
from ionbeam.core.config_parser import parse_config
from ionbeam.core.bases import *
from ionbeam.core.source import Source
from IPython.display import display
from ionbeam.metadata.db import init_db
import sqlalchemy
from datetime import datetime, UTC, timedelta

# init_db(config.globals) 

In [6]:
from copy import deepcopy
from ionbeam.core.bases import FinishMessage, FileMessage, MetaData
from ionbeam.aggregators import TimeAggregator
from ionbeam.core.singleprocess_pipeline import process_message
from datetime import datetime, timedelta

config_file = Path("~/git/IonBeam_bundle/IonBeam/config/").expanduser()
config, actions = parse_config(config_file,
                    environment  = "local",
                    sources = ["smart_citizen_kit"],
                    download = False,
                    die_on_error = True,
                    reingest_from = datetime.fromisoformat("2021-01-01").replace(tzinfo=UTC),
                    )

config.globals.ingestion_time_constants.query_timespan = dataclasses.replace(
    config.globals.ingestion_time_constants.query_timespan,
    start = datetime.now(UTC) - timedelta(hours = 10))

sources, stateless_actions, aggregators = [], [], []
for action in actions:
    if isinstance(action, Source):
        sources.append(action)
    elif isinstance(action, Aggregator):
        aggregators.append(action)
    else:
        stateless_actions.append(action)

online = True
source = sources[0]
incoming_message_stream = source.generate()
input_message = next(incoming_message_stream)

possible_actions = [a for a in actions if not isinstance(a, Source)]
message = deepcopy(input_message)

message_history = []
actions_run = set()
while True:
    display(message)
    message_history.append(deepcopy(message))
    matching = list(message.find_matches(config.globals.actions_by_id))
    
    if not matching: 
        print("No more matches")
        break

    print("That matched with: ", [str(a) for a in matching])
    action = matching[0]
    if action.id in actions_run:
        print(f"Error: loop found for {action} {action.id = }")
        break

    # print(f"current message id {id(message)}")
    message = next(process_message(message, action))
    actions_run.add(action.id)

    
        

DEBUG:ionbeam.core.config_parser.config_parser:Configuration Directory: /Users/math/git/IonBeam_bundle/IonBeam/config
DEBUG:ionbeam.core.config_parser.config_parser:Global config file: /Users/math/git/IonBeam_bundle/IonBeam/config/config.yaml
DEBUG:ionbeam.core.config_parser.config_parser:Overriding global config with {'environment': 'local', 'download': True, 'die_on_error': True, 'reingest_from': datetime.datetime(2021, 1, 1, 0, 0, tzinfo=datetime.timezone.utc)} from command line
DEBUG:ionbeam.core.config_parser.config_parser:Loaded global config...
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.data_path to /Users/math/git/IonBeam_bundle/data
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.cache_path to /Users/math/git/IonBeam_bundle/data/cache
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.fdb_schema_path to /Users/math/git/IonBeam_bundle/IonBeam/config/schema
DEBUG:ionbeam.core.config_parser.config_parser:Resol

TabularMessage(next=None, metadata=MetaData(source_action_id=UUID('f3fb4688-d40e-483c-b2ab-455ebec6537e'), state=None, source=None, observation_variable=None, time_span=TimeSpan(start=datetime.datetime(2025, 1, 19, 11, 0, tzinfo=datetime.timezone.utc), end=datetime.datetime(2025, 1, 19, 12, 0, tzinfo=datetime.timezone.utc)), encoded_format=None, filepath=None, mars_id=None, unstructured={}, columns={'tvoc': RawVariable(name='total_volatile_organic_compounds', description=None, unit='ppb', type=None, CRS=None, dtype='float64', discard=False, key='tvoc', canonical_variable=CanonicalVariable(name='total_volatile_organic_compounds', unit='ppb', desc=None), converter=None, metadata={'id': 113, 'name': 'AMS CCS811 - TVOC', 'tags': [], 'unit': 'ppb', 'uuid': '0c2a1afc-dc08-4066-aacb-0bde6a3ae6f5', 'value': 601.0, 'ancestry': '111', 'datasheet': None, 'created_at': '2019-03-21T16:43:37Z', 'prev_value': 596.0, 'updated_at': '2019-03-21T16:43:37Z', 'default_key': 'tvoc', 'description': 'Total Volatile Organic Compounds Digital Indoor Sensor', 'measurement': {'id': 47, 'name': 'TVOC', 'unit': None, 'uuid': 'c6f9a729-1782-4da1-adc9-e88b7143e45c', 'definition': None, 'description': 'Total volatile organic compounds is a grouping of a wide range of organic chemical compounds to simplify reporting when these are present in ambient air or emissions. Many substances, such as natural gas, could be classified as volatile organic compounds (VOCs).'}, 'last_reading_at': '2025-01-19T11:47:01Z', 'unit_definition': None, 'sensor_key': 'tvoc'}), 'external_station_id': CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), 'eco2': RawVariable(name='equivalent_carbon_dioxide', description=None, unit='ppm', type=None, CRS=None, dtype='float64', discard=False, key='eco2', canonical_variable=CanonicalVariable(name='equivalent_carbon_dioxide', unit='ppm', desc='The equivalent amount of CO2 in the air when the warming effects of other greenhouse gases are taken into account.'), converter=None, metadata={'id': 112, 'name': 'AMS CCS811 - eCO2', 'tags': [], 'unit': 'ppm', 'uuid': '995343c9-12ac-40c0-b6b9-19699e524f86', 'value': 1911.0, 'ancestry': '111', 'datasheet': None, 'created_at': '2019-03-21T16:43:37Z', 'prev_value': 1908.0, 'updated_at': '2019-03-21T16:43:37Z', 'default_key': 'eco2', 'description': 'Equivalent Carbon Dioxide Digital Indoor Sensor', 'measurement': {'id': 46, 'name': 'eCO2', 'unit': None, 'uuid': 'b6fee847-2bb6-4e1e-8e39-979612e2beb9', 'definition': None, 'description': 'Equivalent CO2 is the concentration of CO2 that would cause the same level of radiative forcing as a given type and concentration of greenhouse gas. Examples of such greenhouse gases are methane, perfluorocarbons, and nitrous oxide. CO2 is primarily a by-product of human metabolism and is constantly being emitted into the indoor environment by building occupants. CO2 may come from combustion sources as well. Associations of higher indoor carbon dioxide concentrations with impaired work performance and increased health symptoms have been attributed to correlation of indoor CO2 with concentrations of other indoor air pollutants that are also influenced by rates of outdoor-air ventilation.'}, 'last_reading_at': '2025-01-19T11:47:01Z', 'unit_definition': None, 'sensor_key': 'eco2'}), 'light': RawVariable(name='ambient_light', description=None, unit='lux', type=None, CRS=None, dtype='float64', discard=False, key='light', canonical_variable=CanonicalVariable(name='ambient_light', unit='lux', desc='The intensity of surrounding light.'), converter=None, metadata={'id': 14, 'name': 'BH1730FVC - Light', 'tags': [], 'unit': 'lux', 'uuid': 'ac4234cf-d2b7-4cfa-8765-9f4477e2de5f', 'value': 1015.0, 'ancestry': None, 'datasheet': None, 'created_at': '2015-02-02T18:24:56Z', 'prev_value': 954.0, 'updated_at': '2021-05-07T16:23:50Z', 'default_key': 'light', 'description': 'Digital Ambient Light Sensor', 'measurement':

That matched with:  ['SetConstants(id = ...ea1ef)']


TabularMessage(next=[UUID('34b0484d-d6a2-41fb-80c8-1daf7d32a798')], metadata=MetaData(source_action_id=UUID('f3fb4688-d40e-483c-b2ab-455ebec6537e'), state=None, source=None, observation_variable=None, time_span=TimeSpan(start=datetime.datetime(2025, 1, 19, 11, 0, tzinfo=datetime.timezone.utc), end=datetime.datetime(2025, 1, 19, 12, 0, tzinfo=datetime.timezone.utc)), encoded_format=None, filepath=None, mars_id=None, unstructured={}, columns={'tvoc': RawVariable(name='total_volatile_organic_compounds', description=None, unit='ppb', type=None, CRS=None, dtype='float64', discard=False, key='tvoc', canonical_variable=CanonicalVariable(name='total_volatile_organic_compounds', unit='ppb', desc=None), converter=None, metadata={'id': 113, 'name': 'AMS CCS811 - TVOC', 'tags': [], 'unit': 'ppb', 'uuid': '0c2a1afc-dc08-4066-aacb-0bde6a3ae6f5', 'value': 601.0, 'ancestry': '111', 'datasheet': None, 'created_at': '2019-03-21T16:43:37Z', 'prev_value': 596.0, 'updated_at': '2019-03-21T16:43:37Z', 'default_key': 'tvoc', 'description': 'Total Volatile Organic Compounds Digital Indoor Sensor', 'measurement': {'id': 47, 'name': 'TVOC', 'unit': None, 'uuid': 'c6f9a729-1782-4da1-adc9-e88b7143e45c', 'definition': None, 'description': 'Total volatile organic compounds is a grouping of a wide range of organic chemical compounds to simplify reporting when these are present in ambient air or emissions. Many substances, such as natural gas, could be classified as volatile organic compounds (VOCs).'}, 'last_reading_at': '2025-01-19T11:47:01Z', 'unit_definition': None, 'sensor_key': 'tvoc'}), 'external_station_id': CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), 'eco2': RawVariable(name='equivalent_carbon_dioxide', description=None, unit='ppm', type=None, CRS=None, dtype='float64', discard=False, key='eco2', canonical_variable=CanonicalVariable(name='equivalent_carbon_dioxide', unit='ppm', desc='The equivalent amount of CO2 in the air when the warming effects of other greenhouse gases are taken into account.'), converter=None, metadata={'id': 112, 'name': 'AMS CCS811 - eCO2', 'tags': [], 'unit': 'ppm', 'uuid': '995343c9-12ac-40c0-b6b9-19699e524f86', 'value': 1911.0, 'ancestry': '111', 'datasheet': None, 'created_at': '2019-03-21T16:43:37Z', 'prev_value': 1908.0, 'updated_at': '2019-03-21T16:43:37Z', 'default_key': 'eco2', 'description': 'Equivalent Carbon Dioxide Digital Indoor Sensor', 'measurement': {'id': 46, 'name': 'eCO2', 'unit': None, 'uuid': 'b6fee847-2bb6-4e1e-8e39-979612e2beb9', 'definition': None, 'description': 'Equivalent CO2 is the concentration of CO2 that would cause the same level of radiative forcing as a given type and concentration of greenhouse gas. Examples of such greenhouse gases are methane, perfluorocarbons, and nitrous oxide. CO2 is primarily a by-product of human metabolism and is constantly being emitted into the indoor environment by building occupants. CO2 may come from combustion sources as well. Associations of higher indoor carbon dioxide concentrations with impaired work performance and increased health symptoms have been attributed to correlation of indoor CO2 with concentrations of other indoor air pollutants that are also influenced by rates of outdoor-air ventilation.'}, 'last_reading_at': '2025-01-19T11:47:01Z', 'unit_definition': None, 'sensor_key': 'eco2'}), 'light': RawVariable(name='ambient_light', description=None, unit='lux', type=None, CRS=None, dtype='float64', discard=False, key='light', canonical_variable=CanonicalVariable(name='ambient_light', unit='lux', desc='The intensity of surrounding light.'), converter=None, metadata={'id': 14, 'name': 'BH1730FVC - Light', 'tags': [], 'unit': 'lux', 'uuid': 'ac4234cf-d2b7-4cfa-8765-9f4477e2de5f', 'value': 1015.0, 'ancestry': None, 'datasheet': None, 'created_at': '2015-02-02T18:24:56Z', 'prev_value': 954.0, 'updated_at': '2021-05-07T16:23:50Z', 'default_key': 'light', 'description': 'Dig

That matched with:  ['ComputeStationId(id = ...2a798)']


...

BytesMessage(next=[UUID('44318eb3-7d95-45d5-bfe0-803f9e05b57f')], metadata=MetaData(source_action_id=UUID('880911c4-620c-4fd7-9a75-cbe6e207c5ee'), state='encoded', source=None, observation_variable=None, time_span=TimeSpan(start=datetime.datetime(2025, 1, 19, 11, 0, tzinfo=datetime.timezone.utc), end=datetime.datetime(2025, 1, 19, 12, 0, tzinfo=datetime.timezone.utc)), encoded_format='parquet', filepath=None, mars_id={'class': Key(key='class', value='rd', key_spec=class, reason='Matches', odb_table=None), 'expver': Key(key='expver', value='xxxx', key_spec=expver, reason='Matches', odb_table=None), 'stream': Key(key='stream', value='lwda', key_spec=stream, reason='Matches', odb_table=None), 'aggregation_type': Key(key='aggregation_type', value='by_time', key_spec=aggregation_type=by_time, reason='Matches', odb_table=None), 'platform': Key(key='platform', value='smart_citizen_kit', key_spec=platform, reason='Matches', odb_table=None), 'date': Key(key='date', value='20250119', key_spec=date, reason='Matches', odb_table=None), 'time': Key(key='time', value=datetime.time(11, 0), key_spec=time, reason='Matches', odb_table=None), 'version': Key(key='version', value='1', key_spec=version, reason='Matches', odb_table=None)}, unstructured={}, columns={'total_volatile_organic_compounds': CanonicalVariable(name='total_volatile_organic_compounds', unit='ppb', desc=None), 'external_station_id': CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), 'equivalent_carbon_dioxide': CanonicalVariable(name='equivalent_carbon_dioxide', unit='ppm', desc='The equivalent amount of CO2 in the air when the warming effects of other greenhouse gases are taken into account.'), 'ambient_light': CanonicalVariable(name='ambient_light', unit='lux', desc='The intensity of surrounding light.'), 'noise_dBA': CanonicalVariable(name='noise_dBA', unit='dBA', desc='The noise level in decibels, A-weighted.'), 'air_pressure_near_surface': CanonicalVariable(name='air_pressure_near_surface', unit='Pa', desc=None), 'particulate_matter_1': CanonicalVariable(name='particulate_matter_1', unit='μg/m3', desc=None), 'particulate_matter_10': CanonicalVariable(name='particulate_matter_10', unit='μg/m3', desc=None), 'particulate_matter_2_5': CanonicalVariable(name='particulate_matter_2_5', unit='μg/m3', desc=None), 'carbon_dioxide': CanonicalVariable(name='carbon_dioxide', unit='ppm', desc=None), 'relative_humidity_near_surface': CanonicalVariable(name='relative_humidity_near_surface', unit='%', desc='The relative humidity of the air near the surface.'), 'air_temperature_near_surface': CanonicalVariable(name='air_temperature_near_surface', unit='K', desc='The temperature of the air near the surface.'), 'station_name': CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), 'lat': CanonicalVariable(name='lat', unit='EPSG:4326', desc='The lattitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'lon': CanonicalVariable(name='lon', unit='EPSG:4326', desc='The longitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'author': CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), 'station_created_at': CanonicalVariable(name='station_created_at', unit=None, desc='The date and time that the station was created.'), 'platform': CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), 'aggregation_type': CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), 'station_id': CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), 'chunk_date': CanonicalVariable(name='chunk_date', unit=None, desc='The date of the chunk of data in the format YYYYMMDD'), 'chunk_time': CanonicalVariable(name='chunk_time', unit=None, desc='T

DEBUG:ionbeam.writers.fdb_writer:Found 0 existing matches for {'class': 'rd', 'expver': 'xxxx', 'stream': 'lwda', 'aggregation_type': 'by_time', 'platform': 'smart_citizen_kit', 'date': '20250119', 'time': '1100', 'version': '1'} in 35.41 ms
DEBUG:ionbeam.writers.fdb_writer:Added new data for {'class': 'rd', 'expver': 'xxxx', 'stream': 'lwda', 'aggregation_type': 'by_time', 'platform': 'smart_citizen_kit', 'date': '20250119', 'time': '1100', 'version': '1'} in 6.20 ms


That matched with:  ['FDBWriter()']


DataMessage(next=None, metadata=MetaData(source_action_id=UUID('44318eb3-7d95-45d5-bfe0-803f9e05b57f'), state='written', source=None, observation_variable=None, time_span=TimeSpan(start=datetime.datetime(2025, 1, 19, 11, 0, tzinfo=datetime.timezone.utc), end=datetime.datetime(2025, 1, 19, 12, 0, tzinfo=datetime.timezone.utc)), encoded_format='parquet', filepath=None, mars_id={'class': Key(key='class', value='rd', key_spec=class, reason='Matches', odb_table=None), 'expver': Key(key='expver', value='xxxx', key_spec=expver, reason='Matches', odb_table=None), 'stream': Key(key='stream', value='lwda', key_spec=stream, reason='Matches', odb_table=None), 'aggregation_type': Key(key='aggregation_type', value='by_time', key_spec=aggregation_type=by_time, reason='Matches', odb_table=None), 'platform': Key(key='platform', value='smart_citizen_kit', key_spec=platform, reason='Matches', odb_table=None), 'date': Key(key='date', value='20250119', key_spec=date, reason='Matches', odb_table=None), 'time': Key(key='time', value=datetime.time(11, 0), key_spec=time, reason='Matches', odb_table=None), 'version': Key(key='version', value='1', key_spec=version, reason='Matches', odb_table=None)}, unstructured={}, columns={'total_volatile_organic_compounds': CanonicalVariable(name='total_volatile_organic_compounds', unit='ppb', desc=None), 'external_station_id': CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), 'equivalent_carbon_dioxide': CanonicalVariable(name='equivalent_carbon_dioxide', unit='ppm', desc='The equivalent amount of CO2 in the air when the warming effects of other greenhouse gases are taken into account.'), 'ambient_light': CanonicalVariable(name='ambient_light', unit='lux', desc='The intensity of surrounding light.'), 'noise_dBA': CanonicalVariable(name='noise_dBA', unit='dBA', desc='The noise level in decibels, A-weighted.'), 'air_pressure_near_surface': CanonicalVariable(name='air_pressure_near_surface', unit='Pa', desc=None), 'particulate_matter_1': CanonicalVariable(name='particulate_matter_1', unit='μg/m3', desc=None), 'particulate_matter_10': CanonicalVariable(name='particulate_matter_10', unit='μg/m3', desc=None), 'particulate_matter_2_5': CanonicalVariable(name='particulate_matter_2_5', unit='μg/m3', desc=None), 'carbon_dioxide': CanonicalVariable(name='carbon_dioxide', unit='ppm', desc=None), 'relative_humidity_near_surface': CanonicalVariable(name='relative_humidity_near_surface', unit='%', desc='The relative humidity of the air near the surface.'), 'air_temperature_near_surface': CanonicalVariable(name='air_temperature_near_surface', unit='K', desc='The temperature of the air near the surface.'), 'station_name': CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), 'lat': CanonicalVariable(name='lat', unit='EPSG:4326', desc='The lattitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'lon': CanonicalVariable(name='lon', unit='EPSG:4326', desc='The longitude of the observation, referenced to WGS84 (EPSG: 4326)'), 'author': CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), 'station_created_at': CanonicalVariable(name='station_created_at', unit=None, desc='The date and time that the station was created.'), 'platform': CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), 'aggregation_type': CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), 'station_id': CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), 'chunk_date': CanonicalVariable(name='chunk_date', unit=None, desc='The date of the chunk of data in the format YYYYMMDD'), 'chunk_time': CanonicalVariable(name='chunk_time', unit=None, desc='The time of the chunk of data in the format 

No more matches


In [37]:
from ionbeam.core.singleprocess_pipeline import load_most_recent_error

def print_message_history(m):
    for m in m.history:
        display(m.message)
    

saved_err = load_most_recent_error(config.globals)
# print_message_history(saved_err.message)
saved_err.message

TabularMessage(next=[UUID('cccefccb-bb9c-4d0f-9726-0d08d10063c5')], metadata=MetaData(source_action_id=UUID('6ea9b7e7-0672-4000-8708-9ef758a2d9ab'), state=None, source=None, observation_variable=None, time_span=TimeSpan(start=datetime.datetime(2025, 1, 19, 12, 0, tzinfo=datetime.timezone.utc), end=datetime.datetime(2025, 1, 19, 13, 0, tzinfo=datetime.timezone.utc)), encoded_format=None, filepath=None, mars_id=None, unstructured={}, columns={'tvoc': RawVariable(name='total_volatile_organic_compounds', description=None, unit='ppb', type=None, CRS=None, dtype='float64', discard=False, key='tvoc', canonical_variable=CanonicalVariable(name='total_volatile_organic_compounds', unit='ppb', desc=None), converter=None, metadata={'id': 113, 'name': 'AMS CCS811 - TVOC', 'tags': [], 'unit': 'ppb', 'uuid': '0c2a1afc-dc08-4066-aacb-0bde6a3ae6f5', 'value': 2535.0, 'ancestry': '111', 'datasheet': None, 'created_at': '2019-03-21T16:43:37Z', 'prev_value': 2564.0, 'updated_at': '2019-03-21T16:43:37Z', 'default_key': 'tvoc', 'description': 'Total Volatile Organic Compounds Digital Indoor Sensor', 'measurement': {'id': 47, 'name': 'TVOC', 'unit': None, 'uuid': 'c6f9a729-1782-4da1-adc9-e88b7143e45c', 'definition': None, 'description': 'Total volatile organic compounds is a grouping of a wide range of organic chemical compounds to simplify reporting when these are present in ambient air or emissions. Many substances, such as natural gas, could be classified as volatile organic compounds (VOCs).'}, 'last_reading_at': '2025-01-19T11:41:28Z', 'unit_definition': None, 'sensor_key': 'tvoc'}), 'external_station_id': CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), 'eco2': RawVariable(name='equivalent_carbon_dioxide', description=None, unit='ppm', type=None, CRS=None, dtype='float64', discard=False, key='eco2', canonical_variable=CanonicalVariable(name='equivalent_carbon_dioxide', unit='ppm', desc='The equivalent amount of CO2 in the air when the warming effects of other greenhouse gases are taken into account.'), converter=None, metadata={'id': 112, 'name': 'AMS CCS811 - eCO2', 'tags': [], 'unit': 'ppm', 'uuid': '995343c9-12ac-40c0-b6b9-19699e524f86', 'value': 2845.0, 'ancestry': '111', 'datasheet': None, 'created_at': '2019-03-21T16:43:37Z', 'prev_value': 2859.0, 'updated_at': '2019-03-21T16:43:37Z', 'default_key': 'eco2', 'description': 'Equivalent Carbon Dioxide Digital Indoor Sensor', 'measurement': {'id': 46, 'name': 'eCO2', 'unit': None, 'uuid': 'b6fee847-2bb6-4e1e-8e39-979612e2beb9', 'definition': None, 'description': 'Equivalent CO2 is the concentration of CO2 that would cause the same level of radiative forcing as a given type and concentration of greenhouse gas. Examples of such greenhouse gases are methane, perfluorocarbons, and nitrous oxide. CO2 is primarily a by-product of human metabolism and is constantly being emitted into the indoor environment by building occupants. CO2 may come from combustion sources as well. Associations of higher indoor carbon dioxide concentrations with impaired work performance and increased health symptoms have been attributed to correlation of indoor CO2 with concentrations of other indoor air pollutants that are also influenced by rates of outdoor-air ventilation.'}, 'last_reading_at': '2025-01-19T11:41:28Z', 'unit_definition': None, 'sensor_key': 'eco2'}), 'light': RawVariable(name='ambient_light', description=None, unit='lux', type=None, CRS=None, dtype='float64', discard=False, key='light', canonical_variable=CanonicalVariable(name='ambient_light', unit='lux', desc='The intensity of surrounding light.'), converter=None, metadata={'id': 14, 'name': 'BH1730FVC - Light', 'tags': [], 'unit': 'lux', 'uuid': 'ac4234cf-d2b7-4cfa-8765-9f4477e2de5f', 'value': 12.0, 'ancestry': None, 'datasheet': None, 'created_at': '2015-02-02T18:24:56Z', 'prev_value': 11.0, 'updated_at': '2021-05-07T16:23:50Z', 'default_key': 'light', 'description': 'Digi

In [7]:
from uuid import UUID

id2action = {a.id : a for a in actions}
sources = [a for a in actions if isinstance(a, Source)]
processors = [a for a in actions if not isinstance(a, Source)]

# chains = [[s,] for s in sources]
# for a in processors:
#     if not isinstance(a.match, UUID): 
#         chains.append([str(a.match), a,])
#         continue
#     for c in chains:
#         matches = a.match.int == c[-1].id.int
#         if matches: c.append(a)
            

# for c in chains:
#     print(" --> ".join(str(a) for a in c))

In [8]:
sources, stateless_actions, aggregators = [], [], []
for action in actions:
    if isinstance(action, Source):
        sources.append(action)
    elif isinstance(action, Aggregator):
        aggregators.append(action)
    else:
        stateless_actions.append(action)

print("Sources")
for i, a in enumerate(sources):
    display(a)

print("Aggregators")
for i, a in enumerate(aggregators):
    display(a)

print("Actions")
for i, a in enumerate(stateless_actions):
    display(a)

Sources


name,None
forward_to_names,None
id,f3fb4688-d40e-483c-b2ab-455ebec6537e
next,[SetConstants(id = ...ea1ef)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/smart_citizen_kit/actions.yaml
copy_metadata_to_columns,"{'external_station_id': 'device.id', 'station_name': 'device.name', 'lat': 'device.location.latitude', 'lon': 'device.location.longitude', 'author': 'device.owner.username', 'station_created_at': 'device.created_at'}"
finish_after,None
cache_version,3
use_cache,True
source,smart_citizen_kit
maximum_request_size,"1 day, 0:00:00"


Aggregators
Actions


name,None
forward_to_names,None
id,2457f48d-4e6b-4d85-9fc2-6877539ea1ef
next,[ComputeStationId(id = ...2a798)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/smart_citizen_kit/actions.yaml
match,f3fb4688-d40e-483c-b2ab-455ebec6537e
set,"{'platform': 'smart_citizen_kit', 'aggregation_type': 'by_time'}"
name,value
source_action_id,2457f48d-4e6b-4d85-9fc2-6877539ea1ef
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was..."


name,None
forward_to_names,None
id,34b0484d-d6a2-41fb-80c8-1daf7d32a798
next,[FormatChecks(id = ...48429)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/smart_citizen_kit/actions.yaml
match,2457f48d-4e6b-4d85-9fc2-6877539ea1ef
name,value
source_action_id,34b0484d-d6a2-41fb-80c8-1daf7d32a798
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was..."
canonical_variables_by_name,"{'platform': CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), 'living_lab': CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), 'aggregation_type': CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), 'author': CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), 'external_station_id': CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), 'station_id': CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), 'station_name': CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), 'sensor_name': CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor ..."


...

name,None
forward_to_names,None
id,c1d99775-be04-4e42-a196-732eef04f5c8
next,[ComputeMARSIdentifier(id = ...7034b)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/smart_citizen_kit/actions.yaml
match,878e9837-496d-4b7b-ba5b-e8f8c7200280
name,value
source_action_id,c1d99775-be04-4e42-a196-732eef04f5c8
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was..."
canonical_variables_by_name,"{'platform': CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), 'living_lab': CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), 'aggregation_type': CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), 'author': CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), 'external_station_id': CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), 'station_id': CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), 'station_name': CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), 'sensor_name': CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor ..."


name,None
forward_to_names,[final_processing]
id,ce17d2d0-03ce-49b6-a528-418c43c7034b
next,[IdentityAction(id = ...a5176)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/smart_citizen_kit/actions.yaml
match,c1d99775-be04-4e42-a196-732eef04f5c8
lookup,"{'class': 'const.rd', 'expver': 'const.xxxx', 'stream': 'const.lwda', 'aggregation_type': 'data.aggregation_type', 'platform': 'data.platform', 'date': 'data.chunk_date', 'time': 'data.chunk_time', 'version': 'const.1'}"
name,value
source_action_id,ce17d2d0-03ce-49b6-a528-418c43c7034b
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was..."


name,final_processing
forward_to_names,None
id,6d7944d7-fcde-442d-8071-b2a0030a5176
next,[ParquetEncoder(id = ...7c5ee)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/actions.yaml
match,None
name,value
source_action_id,6d7944d7-fcde-442d-8071-b2a0030a5176
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was..."
canonical_variables_by_name,"{'platform': CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), 'living_lab': CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), 'aggregation_type': CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), 'author': CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), 'external_station_id': CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), 'station_id': CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), 'station_name': CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), 'sensor_name': CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor ..."


name,None
forward_to_names,None
id,880911c4-620c-4fd7-9a75-cbe6e207c5ee
next,[FDBWriter()]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/actions.yaml
match,6d7944d7-fcde-442d-8071-b2a0030a5176
name,value
source_action_id,880911c4-620c-4fd7-9a75-cbe6e207c5ee
state,encoded
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation'), CanonicalVariable(name='living_lab', unit=None, desc='The living lab associated with this data'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. by_time or by_station'), CanonicalVariable(name='author', unit=None, desc='The person or entity responsible for the observation.'), CanonicalVariable(name='external_station_id', unit=None, desc='The identifier for a station used by the observation platform.'), CanonicalVariable(name='station_id', unit=None, desc='A unique identifer for a station, sensor or device.'), CanonicalVariable(name='station_name', unit=None, desc='A human readable name for a station or device. Not necessarily unique.'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was..."


name,None
forward_to_names,None
id,44318eb3-7d95-45d5-bfe0-803f9e05b57f
next,None
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/actions.yaml
match,880911c4-620c-4fd7-9a75-cbe6e207c5ee
config,"{'engine': 'toc', 'spaces': [{'handler': 'Default', 'roots': [{'path': '/Users/math/git/IonBeam_bundle/data/fdb_root'}]}], 'schema': '/Users/math/git/IonBeam_bundle/IonBeam/config/schema'}"
debug,[]
name,value
source_action_id,44318eb3-7d95-45d5-bfe0-803f9e05b57f
state,written


In [9]:
# time_span = TimeSpan.last(days = 50)

# datastreams = list(source.get_data_streams(time_span))
# datastreams

In [47]:
from ionbeam.metadata import db

# Delete sessions with null timespans no idea how that happened
with Session(config.globals.sql_engine) as db_session:
    for s in db_session.query(db.Station).all():
        if not s._time_span:
            db_session.delete(s)
            db_session.commit()
            print(s.platform)
            print(s.location)
            print(s.authors)
            
    
    # for ds in datastreams:
    #     existing_chunks = ds.get_chunks(db_session, time_span)
    #     print(list(existing_chunks))
    

In [11]:
# from matplotlib import pyplot as plt
# from datetime import UTC

# show_time_span = TimeSpan.last(days = 100)

# def plot_datastreams(data_streams):

#     n = len(data_streams)
#     f, axes = plt.subplots(n, 1, figsize = (10, 2*n), sharex = True, squeeze = False)
    
#     def plot_timespan(ax, ts, colour, label = None):
#         ax.axvline(ts.start, linestyle = "dotted", color = colour)
#         ax.axvline(ts.end, linestyle = "solid", color = colour)
#         ax.axvspan(ts.start, ts.end, alpha = 0.1, color = colour, label = label)
    
#     with Session(config.globals.sql_engine) as db_session:
#         for ds, ax in zip(data_streams, axes[0]):
#             existing_chunks = ds.get_chunks(db_session, show_time_span)
            
#             for i, chunk in enumerate(existing_chunks):
#                 if not chunk.success:
#                     pprint(chunk)
#                 colour = "red" if not chunk.success else "green"
#                 label = "Failed" if not chunk.success else "Sucess"
#                 plot_timespan(ax, chunk.time_span, colour, label = label)

#                 times = set(datetime.strptime(t, "%Y%m%d%H%M")
#                             for readings in chunk.json
#                             for t in readings["timeline"])
#                 ax.scatter(y = [i%2 for _ in times],  x = list(times), color = "blue")
            
#             gaps = source.gaps_in_database(db_session, ds, show_time_span)
#             for ts in gaps: 
#                 # print(f"gap: {ts.delta()}")
#                 plot_timespan(ax, ts, "blue", label = "Todo")
    
#             ax.axvline(datetime.now(UTC), linestyle = "dotted", color = "k")
#             ax.set(ylabel = ds.key)
#     # ax.legend()

# for source in sources:
#     print(f"Source: {source}")
#     plot_datastreams(list(source.get_data_streams(show_time_span)))

In [12]:
message_history[0].data

sensor_key,external_station_id,bar,bat,co2,eco2,ext_h,ext_t,h,light,noise_dba,pm_avg_1,pm_avg_10,pm_avg_2.5,t,tvoc,station_name,lat,lon,author,station_created_at
datetime,,,,,,,,,,,,,,,,,,,,
2025-01-19 11:00:22+00:00,16643,101.13,99.0,422.0,1799.0,45.26,16.07,52.77,22.0,63.38,NaN,NaN,NaN,13.08,448.0,I-CHANGE CIMA CASTAGNA,44.49617,8.92065,smart_citizen_kit,2023-10-09 06:37:25+00:00
2025-01-19 11:00:22+00:00,16774,102.16,100.0,407.0,2732.0,41.69,16.12,53.06,9.0,54.57,NaN,NaN,NaN,12.82,2296.0,I-CHANGE CHIAVARI,44.32045,9.32365,smart_citizen_kit,2023-11-08 17:08:23+00:00
2025-01-19 11:00:26+00:00,16517,0.00,100.0,387.0,7043.0,56.43,11.08,66.41,6.0,61.25,NaN,NaN,NaN,7.55,11484.0,I-CHANGE CIMA ARENZANO,44.39930,8.67264,smart_citizen_kit,2023-07-16 21:52:48+00:00
2025-01-19 11:01:22+00:00,16643,101.13,99.0,423.0,1799.0,45.37,16.05,52.86,22.0,57.78,2.0,7.0,7.0,13.08,448.0,I-CHANGE CIMA CASTAGNA,44.49617,8.92065,smart_citizen_kit,2023-10-09 06:37:25+00:00
2025-01-19 11:01:25+00:00,16774,102.16,99.0,406.0,2742.0,41.79,16.12,53.08,9.0,57.09,0.0,1.0,0.0,12.82,2309.0,I-CHANGE CHIAVARI,44.32045,9.32365,smart_citizen_kit,2023-11-08 17:08:23+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-19 11:58:31+00:00,16774,102.13,100.0,409.0,2872.0,41.18,16.42,52.24,14.0,53.56,NaN,NaN,NaN,13.22,2589.0,I-CHANGE CHIAVARI,44.32045,9.32365,smart_citizen_kit,2023-11-08 17:08:23+00:00
2025-01-19 11:58:36+00:00,16517,0.00,100.0,385.0,7199.0,58.88,10.81,69.26,6.0,53.81,NaN,NaN,NaN,7.24,11807.0,I-CHANGE CIMA ARENZANO,44.39930,8.67264,smart_citizen_kit,2023-07-16 21:52:48+00:00
2025-01-19 11:59:25+00:00,16643,101.07,98.0,418.0,1860.0,48.92,14.71,56.54,14.0,60.21,NaN,NaN,NaN,11.76,529.0,I-CHANGE CIMA CASTAGNA,44.49617,8.92065,smart_citizen_kit,2023-10-09 06:37:25+00:00


## Reload and retry failed action

In [13]:
from ionbeam.core.config_parser import reload_action
from IPython.display import display

print(f"Failed action: {action} Definition: {action.definition_path}")

config, action = reload_action(config, action)

message = next(action.process(message_history[-1]))

display(action)
display(message)

DEBUG:ionbeam.core.config_parser.config_parser:Reloading FDBWriter() from /Users/math/git/IonBeam_bundle/IonBeam/config/actions.yaml
DEBUG:ionbeam.core.config_parser.config_parser:Configuration Directory: /Users/math/git/IonBeam_bundle/IonBeam/config
DEBUG:ionbeam.core.config_parser.config_parser:Global config file: /Users/math/git/IonBeam_bundle/IonBeam/config/config.yaml


Failed action: FDBWriter() Definition: /Users/math/git/IonBeam_bundle/IonBeam/config/actions.yaml


DEBUG:ionbeam.core.config_parser.config_parser:Overriding global config with {} from command line
DEBUG:ionbeam.core.config_parser.config_parser:Loaded global config...
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.data_path to /Users/math/git/IonBeam_bundle/data
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.cache_path to /Users/math/git/IonBeam_bundle/data/cache
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.fdb_schema_path to /Users/math/git/IonBeam_bundle/IonBeam/config/schema
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.fdb_root to /Users/math/git/IonBeam_bundle/data/fdb_root
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.metkit_language_template to /Users/math/git/IonBeam_bundle/IonBeam/config/language.yaml.template
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.secrets_file to /Users/math/git/IonBeam_bundle/IonBeam/config/secrets.yaml

AttributeError: 'DataMessage' object has no attribute 'data_bytes'

In [ ]:
m = message_history[-1]
m.metadata.unstructured

In [ ]:
import pyodc as pyodc
from io import BytesIO

f = BytesIO(message.bytes)
df = pyodc.read_odb(f, single = True)
df

In [ ]:
from ionbeam.core.converters import unit_conversions

df = pd.DataFrame({"A" : np.arange(100), "B": np.linspace(0,1,100)})

In [ ]:
converter = unit_conversions["°C -> K"]


%timeit df.apply({"A" : converter}, raw = True, by_row = False)

In [ ]:
converter = lambda t: t + 273.15
converter2 = lambda t: t * 1000

df = pd.DataFrame({"A" : np.arange(100), "B": np.linspace(0,1,100), "C" : np.arange(100)})

df = df.apply({"A" : converter, "B" : converter2}, raw = True, by_row = False)

df

In [ ]:
converter = lambda 